In [1]:
import pandas as pd
from io import StringIO
import re
import os
import ast
import pandas as pd
import tqdm
pd.options.mode.chained_assignment = None  # default='warn'

base = "D://hidden_cve/"

import json
with open(f'D:/hidden_cve/rawdata/repo_commits.json', 'r') as fin:
    repo_commits = json.load(fin)

In [2]:
filename = os.listdir(base+'graphql')[1634]

In [30]:
def find_name(name):
    for key in repo_commits.keys():
        if key.endswith("/"+name):
            return key
    return None

# Get All data from graphql
## new_dfs - list of all events from all repos in graphql (except for commit data)
cols = ['vulnerabilityAlerts', 'forks', 'issues', 'pullRequests', 'releases', 'stargazers']

gql_dfs = []
commit_dict = dict()
for filename in tqdm.tqdm(os.listdir("D:/hidden_cve/graphql3/")):
    df = pd.read_csv(f"D:/hidden_cve/graphql3/{filename}")
    cur_repo_name = filename.split(".csv")[0]
    if df.empty:
        continue
    
    commits = df[["additions","deletions","commit_date"]]
    commits['created_at']=pd.to_datetime(df['commit_date'])
    commits = commits.drop(["commit_date"],axis=1)
    try:
        # day_df = commits.resample("D",on="created_at").sum()
        day_df = commits.set_index("created_at")
    except (AttributeError,KeyError) as e:
        continue

    commit_dict[cur_repo_name]=day_df
    if cur_repo_name is None:
        print("Find name error ", name)
        continue
    for col in cols:
        
        if col in df.columns and not df[col].isnull().all():
            cur_df = pd.DataFrame()
            cur_df['created_at'] = pd.to_datetime(df[col].dropna())
            cur_df["type"]=col
            cur_df["name"]= cur_repo_name.replace("/","_")
            gql_dfs.append(cur_df)
            

  7%|█████▊                                                                         | 149/2013 [00:07<01:02, 29.62it/s]C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
 10%|███████▋                                                                       | 196/2013 [00:10<01:12, 25.10it/s]C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
 12%|█████████▌                                                                     | 245/2013 [00:12<01:24, 20.98it/s]C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,3,5) have mixed types.Specify dt

In [31]:
## Vuln_lst = list of vulnerabilities per repo
## ga_dfs github archive events

vuln_lst = []
ga_dfs = []
for year in tqdm.tqdm(range(2015,2022)):
    df = pd.read_csv(f'D:/repo_data5/{year}.csv')
    df.name = df.name.str.replace("/","_")
    ga_dfs.append(df)
for repo,dates in tqdm.tqdm(list(repo_commits.items())):
    repo =  repo.replace("/","_")
    for date in dates:
        vuln_lst.append(("VulnEvent",repo,date[1]))
ga_dfs.append(pd.DataFrame(vuln_lst,columns = ga_dfs[0].columns))
all_df = pd.concat(ga_dfs+gql_dfs)
repo_list =  [repo for repo in all_df.groupby('name')]

100%|██████████████████████████████████████████████████████████████████████████| 2311/2311 [00:00<00:00, 252357.11it/s]


In [32]:
def add_commits(day_df,commit_dict,repo_name):
    try:
        return day_df.join(commit_dict[repo_name],how='outer')
    except KeyError:
        return None
    
"""try:
    df = pd.read_csv(f"D:/hidden_cve/graphql3/{repo_name}.csv")
    if df.empty:
        return day_df

    commits = df[["additions","deletions","commit_date"]]
    commits['created_at']=pd.to_datetime(df['commit_date'])
    commits = commits.drop(["commit_date"],axis=1)
    day_df = commits.resample("D",on="created_at").sum()
    return day_df
except(KeyError,FileNotFoundError):
    return None"""


'try:\n    df = pd.read_csv(f"D:/hidden_cve/graphql3/{repo_name}.csv")\n    if df.empty:\n        return day_df\n\n    commits = df[["additions","deletions","commit_date"]]\n    commits[\'created_at\']=pd.to_datetime(df[\'commit_date\'])\n    commits = commits.drop(["commit_date"],axis=1)\n    day_df = commits.resample("D",on="created_at").sum()\n    return day_df\nexcept(KeyError,FileNotFoundError):\n    return None'

In [42]:
commit_dict[repo_name].index = pd.to_datetime(commit_dict[repo_name].index).tz_localize('Etc/UCT')

day_df.join(commit_dict[repo_name],how='outer')



,WatchEvent,stargazers,forks,IssueCommentEvent,pullRequests,PushEvent,PullRequestEvent,IssuesEvent,issues,VulnEvent,CreateEvent,ForkEvent,additions,deletions
created_at,,,,,,,,,,,,,,
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
mlst = ['CreateEvent', 'ForkEvent', 'GollumEvent',
       'IssueCommentEvent', 'IssuesEvent', 'MemberEvent', 'PullRequestEvent',
       'PushEvent', 'ReleaseEvent', 'VulnEvent', 'WatchEvent', 'forks',
       'issues', 'pullRequests', 'releases', 'stargazers',
       'vulnerabilityAlerts']


all_columns = list(set(list(all_df.type.unique()) +mlst))
print(len(all_columns),all_columns)
repo_dfs = []
i = 0
err_list = open("D:/errlist.txt",'a')
for repo_name,repo_df in repo_list:
    repo_name =  repo_name.replace("/","_")
    if repo_df[(repo_df.type == 'VulnEvent')].empty:
        err_list.write(f"{repo_name}\n")
        continue
    day_df = pd.DataFrame()
    empty_cols = []
    for col in all_columns:
        cur_type = repo_df[(repo_df.type == col)]
        if cur_type.empty:
            empty_cols.append(col)
            continue
        cur_df = pd.to_datetime(cur_type.created_at)
        cur_df = pd.DataFrame(cur_df).set_index('created_at')
        cur_df[col]=1
        # cur_df = cur_df.resample("D").sum()
        day_df = day_df.join(cur_df,how='outer')
        day_df = day_df.fillna(0)
        
    # Adding commits
    try:
        day_df = add_commits(day_df,commit_dict,repo_name)
    except TypeError:
        print(repo_name)
        continue
    if day_df is None:
        err_list.write(repo_name+"\n")
        continue
    for col in empty_cols:
        day_df[col]=0
    print(repo_name,len(empty_cols))
    day_df = day_df.reindex(sorted(day_df.columns), axis=1)
    day_df = day_df.fillna(0)
    day_df.to_csv(f"D:/repo_gharchive_processed4/{repo_name.replace('/','_')}.csv")
err_list.close()

22 ['WatchEvent', 'MemberEvent', 'GollumEvent', 'ReleaseEvent', 'stargazers', 'forks', 'PublicEvent', 'PullRequestReviewEvent', 'PullRequestReviewCommentEvent', 'CommitCommentEvent', 'IssueCommentEvent', 'releases', 'pullRequests', 'PushEvent', 'PullRequestEvent', 'IssuesEvent', 'vulnerabilityAlerts', 'issues', 'DeleteEvent', 'VulnEvent', 'CreateEvent', 'ForkEvent']
01org_opa-ff 6
01org_opa-fm 6
01org_tpm2.0-tools 2
10gen-archive_mongo-c-driver-legacy 15
1up-lab_oneupuploaderbundle 17
389ds_389-ds-base 4
3s3s_opentrade 6
94fzb_zrlog 9
LibVNC_libvncserver 3
aaron-junker_usoc 16
aaugustin_websockets 5
aawc_unrar 6
abcprintf_upload-image-with-ajax 15
abhinavsingh_proxy.py 3
abrt_abrt 2
abrt_libreport 2
academysoftwarefoundation_openexr 16
acassen_keepalived 5
accel-ppp_accel-ppp 8
accenture_mercury 17
acinq_eclair 16
acossette_pillow 11
acpica_acpica 6
actions_http-client 5
adaltas_node-csv-parse 6
adaltas_node-mixme 14
adamghill_django-unicorn 3
adamhathcock_sharpcompress 3
adaptivecompu

In [ ]:
from matplotlib import pyplot
import matplotlib
import os

for csv in tqdm.tqdm(os.listdir("D:/repo_gharchive_processed4")):
    if "errlist" in csv:
        continue
    df = pd.read_csv(f"D:/repo_gharchive_processed4/{csv}",index_col="created_at")
    try:
        idx = list(df.columns).index("VulnEvent")+1
    except ValueError:
        continue
    reds =  [matplotlib.cm.get_cmap('Spectral')(i/len(df.columns)) for i in range(len(df.columns))]
    colors = []
    for i in range(idx-1):
        colors += [reds[i]]
    colors+=["blue"]
    for i in range((len(df.columns)-idx)):
        colors += [reds[i+idx]]
    
    plot = df.drop(["additions","deletions"],axis=1).plot(figsize=(8,8),color=colors,rot=45,title=csv)
    fig = plot.get_figure()
    fig.savefig(f"D:/cve/images/initial2/{csv}.png")

    plot = df[["additions","deletions"]].plot(figsize=(8,8),rot=45,title=csv)
    fig = plot.get_figure()
    fig.savefig(f"D:/cve/images/commits/{csv}.png")
    


